### Generate test matrix

In [1]:
using BenchmarkTools, IterativeSolvers, LinearAlgebra, MatrixDepot, Random

Random.seed!(280)

n = 100
# Poisson matrix of dimension n^2=10000, pd and sparse
A = matrixdepot("poisson", n)
@show typeof(A)
# dense matrix representation of A
Afull = convert(Matrix, A)
@show typeof(Afull)
# sparsity level
count(!iszero, A) / length(A)
b = randn(n^2)

┌ Info: Recompiling stale cache file C:\Users\cobook\.julia\compiled\v1.2\BenchmarkTools\ZXPQo.ji for BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1240


include group.jl for user defined matrix generators
verify download of index files...
used remote site is https://sparse.tamu.edu/?per_page=All
populating internal database...
typeof(A) = SparseArrays.SparseMatrixCSC{Float64,Int64}
typeof(Afull) = Array{Float64,2}


10000-element Array{Float64,1}:
  0.12623784496408763 
 -2.3468794813834966  
  1.9166090582536102  
 -0.23920888512829233 
 -0.5784508270929073  
 -1.1278278196026448  
  1.1478628422667982  
 -1.354705173870581   
 -0.23706547458394342 
 -0.6809935026697     
 -0.8826696457022515  
  1.7138424693341203  
 -0.7339523682572253  
  ⋮                   
 -1.1242756459799825  
 -1.5840165152654142  
 -0.18205667606452444 
 -0.7596266189004605  
 -1.4396141684820316  
  0.004180129114450185
 -0.07636458207051988 
 -1.9658459863745268  
 -1.3457051266647149  
  0.3120272656011855  
  0.6456518202208312  
 -0.28866613765800436 

### Matrix-vector muliplication

### Jacobi solver

It seems that Jacobi solver doesn't give the correct answer.

In [2]:
xjacobi = jacobi(A, b)

10000-element Array{Float64,1}:
 -0.11622985241732496
 -0.4635831908534026 
  0.23226025476409706
 -0.31758303896233386
 -0.5545511535763019 
 -0.4111814236198636 
  0.3181490123608109 
 -0.10506164752145747
 -0.10484671846434306
 -0.34799055652105043
 -0.3266061388021883 
  0.3377951681302126 
 -0.03656658807294293
  ⋮                  
 -0.6028865411307084 
 -0.8572621518011447 
 -0.4292611655954205 
 -0.6022628752430368 
 -0.8685276036611846 
 -0.645085481736975  
 -0.5105671301040766 
 -0.8004520746400694 
 -0.5459042625106357 
  0.0835217275886183 
  0.2886685655652838 
  0.1480671984569923 

[Documentation](https://juliamath.github.io/IterativeSolvers.jl/dev/linear_systems/stationary/#Jacobi-1) reveals that the default value of `maxiter` is 10. A couple trial runs shows that 30000 Jacobi iterations are enough to get an accurate solution.

In [14]:
@benchmark jacobi($A, $b, maxiter=30000)

BenchmarkTools.Trial: 
  memory estimate:  234.72 KiB
  allocs estimate:  9
  --------------
  minimum time:     2.936 s (0.00% GC)
  median time:      3.034 s (0.00% GC)
  mean time:        3.034 s (0.00% GC)
  maximum time:     3.131 s (0.00% GC)
  --------------
  samples:          2
  evals/sample:     1

### Gauss-Seidal

In [27]:
@benchmark gauss_seidel($A, $b, maxiter=15000)

BenchmarkTools.Trial: 
  memory estimate:  156.55 KiB
  allocs estimate:  8
  --------------
  minimum time:     1.477 s (0.00% GC)
  median time:      1.496 s (0.00% GC)
  mean time:        1.493 s (0.00% GC)
  maximum time:     1.502 s (0.00% GC)
  --------------
  samples:          4
  evals/sample:     1

### SOR

In [3]:
xsor = sor(A, b, 0.75, maxiter=10000)

10000-element Array{Float64,1}:
 -0.13573654687466608 
 -0.5206789599457584  
  0.17980466568538686 
 -0.4002791892116248  
 -0.6289282383590915  
 -0.49588500865874097 
  0.22224230893685876 
 -0.21247273236519465 
 -0.2535061468257664  
 -0.5432927433952679  
 -0.592638429470118   
 -0.014784103432608579
 -0.4729947308070689  
  ⋮                   
 -0.7926471240504929  
 -1.2221432286720415  
 -0.9305374922935421  
 -1.1872801056414293  
 -1.4885989790708278  
 -1.2383547534718287  
 -1.0271753757267539  
 -1.2094409837853988  
 -0.8319275353822709  
 -0.09415051921681566 
  0.19629893973017298 
  0.11037395266590884 

In [8]:
norm(xsor - xjacobi)

532.9587839534698

In [15]:
@benchmark sor($A, $b, 0.75, maxiter=10000)

BenchmarkTools.Trial: 
  memory estimate:  547.27 KiB
  allocs estimate:  20010
  --------------
  minimum time:     1.336 s (0.00% GC)
  median time:      1.351 s (0.00% GC)
  mean time:        1.390 s (0.00% GC)
  maximum time:     1.520 s (0.00% GC)
  --------------
  samples:          4
  evals/sample:     1

### Conjugate Gradient

In [28]:
using Distributions

┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1242


In [35]:
ran_vec =rand(Normal(0, 1), 1000)

1000-element Array{Float64,1}:
 -0.9180297412776988 
 -1.7634769653912572 
  0.25443648984172623
 -0.38639775652347746
 -0.48170094071263075
  1.2450785274870282 
 -0.27523837834165477
  1.1113556776790492 
  0.8654377551475115 
  0.6933408873698911 
 -0.23706063661395926
 -0.26957028525662174
 -0.5092907049314676 
  ⋮                  
 -0.2706915171650025 
  0.5240012095319185 
 -0.8702479313741973 
 -0.2828254373967815 
  1.0872866759683317 
 -0.03205966284676451
  1.2237812621626334 
  0.47852700479560545
  0.24852960623192705
  0.9773327102560351 
  0.9245748520867975 
  1.0665133622830874 

In [65]:
LowerTriangular(A)

5×5 LowerTriangular{Float64,Array{Float64,2}}:
  0.387766    ⋅          ⋅          ⋅          ⋅      
 -0.387666   6.64226     ⋅          ⋅          ⋅      
 -0.12516   -0.941094   2.64174     ⋅          ⋅      
  0.694163  -2.34704   -0.401052   1.76654     ⋅      
 -0.140002  -0.694623   0.432363  -0.0764905  0.186699

In [70]:
Diagonal(A).= 1/2 *Diagonal(A)

5×5 Diagonal{Float64,Array{Float64,1}}:
 0.193883   ⋅        ⋅        ⋅        ⋅       
  ⋅        3.32113   ⋅        ⋅        ⋅       
  ⋅         ⋅       1.32087   ⋅        ⋅       
  ⋅         ⋅        ⋅       0.88327   ⋅       
  ⋅         ⋅        ⋅        ⋅       0.0933494

In [71]:
D = copy(Diagonal(A))

5×5 Diagonal{Float64,Array{Float64,1}}:
 0.387766   ⋅        ⋅        ⋅        ⋅      
  ⋅        6.64226   ⋅        ⋅        ⋅      
  ⋅         ⋅       2.64174   ⋅        ⋅      
  ⋅         ⋅        ⋅       1.76654   ⋅      
  ⋅         ⋅        ⋅        ⋅       0.186699

In [73]:
D^(-1)

5×5 Diagonal{Float64,Array{Float64,1}}:
 2.57888   ⋅         ⋅         ⋅         ⋅     
  ⋅       0.150551   ⋅         ⋅         ⋅     
  ⋅        ⋅        0.378538   ⋅         ⋅     
  ⋅        ⋅         ⋅        0.566078   ⋅     
  ⋅        ⋅         ⋅         ⋅        5.35622

http://www.netlib.org/linalg/html_templates/node17.html
![ssor_image](./sub_file/ssor.png)

In [11]:
w/(2-w) * M_sor * D^(-1) * Transpose(M_sor)

10000×10000 Array{Float64,2}:
  5.33333   -0.666667   0.0       …   0.0        0.0        0.0     
 -0.666667   5.41667   -0.666667      0.0        0.0        0.0     
  0.0       -0.666667   5.41667       0.0        0.0        0.0     
  0.0        0.0       -0.666667      0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  0.0        0.0        0.0       …   0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  0.0        0.0        0.0       …   0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  ⋮                               ⋱                                 
  0.

In [10]:
M_sor = (1/w-1) * Diagonal(A) +  LowerTriangular(A)
w/(2-w) * M_sor * D^(-1) * Transpose(M_sor)

10000×10000 LowerTriangular{Float64,Array{Float64,2}}:
  8.0    ⋅     ⋅     ⋅     ⋅     ⋅   …    ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
 -1.0   8.0    ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
  0.0  -1.0   8.0    ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
  0.0   0.0  -1.0   8.0    ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
  0.0   0.0   0.0  -1.0   8.0    ⋅        ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
  0.0   0.0   0.0   0.0  -1.0   8.0  …    ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
  0.0   0.0   0.0   0.0   0.0  -1.0       ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
  0.0   0.0   0.0   0.0   0.0   0.0       ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
  0.0   0.0   0.0   0.0   0.0   0.0       ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
  0.0   0.0   0.0   0.0   0.0   0.0       ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
  0.0   0.0   0.0   0.0   0.0   0.0  …    ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
  0.0   0.0   0.0   0.0   0.0   0.0       ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
  0.0   0.0   0.0   0.0   0.0   0.0       ⋅     ⋅     ⋅  

In [11]:
#Init val
w = 0.5 # not greater than 2?
A_ = copy(A)
D = copy(Diagonal(A))
max_itter = 100
#Diagonal(A_).= 1/w *Diagonal(A_)
#M_sor = LowerTriangular(A_)

#Calc M & N
M_sor = (1/w-1) * D +  LowerTriangular(A)
M_ssor = w/(2-w) * M_sor * D^(-1) * Transpose(M_sor)
N = M_ssor - A


10000×10000 Array{Float64,2}:
 1.33333   0.333333  0.0       0.0       …  0.0       0.0       0.0     
 0.333333  1.41667   0.333333  0.0          0.0       0.0       0.0     
 0.0       0.333333  1.41667   0.333333     0.0       0.0       0.0     
 0.0       0.0       0.333333  1.41667      0.0       0.0       0.0     
 0.0       0.0       0.0       0.333333     0.0       0.0       0.0     
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 ⋮                   

In [12]:
#Calc SSOR
max_itter = 100
γ = sqrt(2/w -1)
D_s = sqrt(D)
y = zeros(length(b))
m_inv = inv(M_ssor)

m_inv_n = m_inv*N
D_s_b = D_s*b
m_inv_D_s_b = γ*m_inv*D_s_b
m_inv_t_n_t = Transpose(m_inv)*Transpose(N)
m_inv_t_D_s_b = γ*Transpose(m_inv)*D_s_b


for i = 1 : max_itter
    print(i)
    x = m_inv_n*y + m_inv_D_s_b
    y = m_inv_t_n_t*x + m_inv_t_D_s_b
    #x = m_inv*(N*y + γ*(D_s*b))
    #y = (m_inv')*(N'x + γ*D_s*b)
end

10000-element Array{Float64,1}:
 -0.08883298053250063
 -1.2434593361184914 
  1.0254074149101564 
 -0.2567539622671509 
 -0.6689801443240143 
 -0.7633226588437758 
  0.7168048560764445 
 -0.6489898940226291 
 -0.265195021695715  
 -0.6043807262245158 
 -0.6038465344161905 
  1.0026544921001344 
 -0.24176817494416905
  ⋮                  
 -0.9086930566762922 
 -1.2665634586360557 
 -0.2343798874088155 
 -0.5877064602698528 
 -1.0450912474431597 
 -0.31692282041693515
 -0.2550672301648231 
 -1.347086260775811  
 -0.9793160003664941 
  0.1876339974682442 
  0.4388798112469138 
  0.01317718103599641

# All solver!


In [27]:
#It takes long time to calc init / inverse matirx 
using SparseArrays


function sor_temp(A,b,w, max_itter)
    batch = 100
    D = sparse(Diagonal(A))
    M = sparse((1/w-1) * D + LowerTriangular(A))
    #N = M - A # automatlly assigned sparse type
    #M_inv_N = sparse(M\N)
    #M_inv_b = sparse(M\b)
    big_itter = max_itter ÷ batch
    x =zeros(length(b))
    r = zeros(length(b))
    #x_pre = zeros(length(b))
    
    for j = 1 : big_itter 
        for i = 1:batch 
            #x[:] = M_inv_N*x + M_inv_b
            r = b - A*x
            x = x + sparse(M\r)
        end
        #내가 봤을때 M_inv_N*x이거 매퍼 써야하는 각
        if norm( r, 2) < 10^-8 
            return x 
        end
    end
    
    return x
end



sor_temp (generic function with 2 methods)

In [29]:
@benchmark sor_temp($A,$b, 0.5,1000) 

BenchmarkTools.Trial: 
  memory estimate:  2.05 GiB
  allocs estimate:  39043
  --------------
  minimum time:     2.726 s (7.86% GC)
  median time:      2.730 s (7.88% GC)
  mean time:        2.730 s (7.88% GC)
  maximum time:     2.735 s (7.90% GC)
  --------------
  samples:          2
  evals/sample:     1

# temp_code

In [ ]:
#Init val
w = 0.5 # not greater than 2?
A_ = copy(A)
D = copy(Diagonal(A))
Diagonal(A_).= 1/w *Diagonal(A_)

#Calc M & N
M_sor = LowerTriangular(A_)
M_ssor = w/(2-w) * M_sor * D^(-1) * M_sor'
N = M_ssor - A

#Calc SSOR
γ = sqrt(2/w -1)
big_itter = itter ÷ batch
D_s = sqrt(D)
for j = 1 : big_itter
    z_1 = rand(Normal(0, 1), big_itter*p)
    z_2 = rand(Normal(0, 1), big_itter*p)
    for i = 1 : batch
        idx = p*batch*(j-1)+p*(i-1)+1
        x = m_inv*(N*y + γ*(D_s*z_1[idx:idx+p-1]))
        y = (m_inv')*(N'x + γ*D_s*z_2[idx:idx+p-1])
    end
    if norm(y-y_pr) < s :
        return y
    end
end


In [34]:
p, big_itter , batch = 3,3,3
hi = 1:p*big_itter*batch
for j = 1: big_itter
    for i = 1: batch
        for k in p*batch*(j-1)+p*(i-1)+1 : p*batch*(j-1)+p*(i-1)+p
            print(k)
        end
    end
end

123456789101112131415161718192021222324252627

# miscellaneous

In [20]:
a1,a2 = findnz(A); 

In [21]:
sqrt(5)

2.23606797749979

In [21]:
[a1[a1 .== a2],a2[a1 .== a2]]

2-element Array{Array{Int64,1},1}:
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 10000]
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 10000]

In [13]:

for i = 1 : max_itter
    print(i)
    x = m_inv_n*y + m_inv_D_s_b
    y = m_inv_t_n_t*x + m_inv_t_D_s_b
    #x = m_inv*(N*y + γ*(D_s*b))
    #y = (m_inv')*(N'x + γ*D_s*b)
end


123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100

In [2]:
using BenchmarkTools, IterativeSolvers, LinearAlgebra, MatrixDepot, Random

Random.seed!(280)

n = 1000
# Poisson matrix of dimension n^2=10000, pd and sparse
A = matrixdepot("poisson", n)
@show typeof(A)
# dense matrix representation of A
#Afull = convert(Matrix, A)
#@show typeof(Afull)
# sparsity level
count(!iszero, A) / length(A)
b = randn(n^2)

typeof(A) = SparseArrays.SparseMatrixCSC{Float64,Int64}


1000000-element Array{Float64,1}:
  0.12623784496408763
 -2.3468794813834966 
  1.9166090582536102 
 -0.23920888512829233
 -0.5784508270929073 
 -1.1278278196026448 
  1.1478628422667982 
 -1.354705173870581  
 -0.23706547458394342
 -0.6809935026697    
 -0.8826696457022515 
  1.7138424693341203 
 -0.7339523682572253 
  ⋮                  
  0.9974336658860118 
 -0.6938258120738878 
 -0.2581456552315425 
 -1.1521788642253536 
  0.9090789430757401 
  0.4155618883658102 
  0.9035113629042111 
  1.0941697035759808 
  0.5103199450199046 
  0.1637036005101834 
  0.24691062073070266
  0.10853338073637239

In [7]:
B = A'

1000000×1000000 Adjoint{Float64,SparseArrays.SparseMatrixCSC{Float64,Int64}}:
  4.0  -1.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   4.0  -1.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   4.0  -1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   4.0  -1.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   4.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   4.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.

In [13]:
using SparseArrays

In [20]:
A = sparse([1.0 2 3 0; 1 1 0 0;0 1 4 1; 1 0 0 10])
Matrix(A)

4×4 Array{Float64,2}:
 1.0  2.0  3.0   0.0
 1.0  1.0  0.0   0.0
 0.0  1.0  4.0   1.0
 1.0  0.0  0.0  10.0

In [19]:
B = sparse(A')
Matrix(B)

4×4 Array{Float64,2}:
 1.0  1.0  0.0   1.0
 2.0  1.0  1.0   0.0
 3.0  0.0  4.0   0.0
 0.0  0.0  1.0  10.0

In [ ]:
[]

need some linear mapper for fast calc